In [ ]:
import pandas as pd

# Cargar datos
df = pd.read_csv("/content/basededatos_visualizacion.csv", sep=";")


# Echar un vistazo
df.head(5)

,Película,Lugar de estreno,País de estreno
0,¿Quien tiene la culpa?,Estreno en salas,Chile
1,Patio de Chacales,Tallinn Black Nights Film Festival,Estonia
2,¿Cuándo te vas?,Estreno en salas,Chile
3,Desconectados 2: Reconectados,Estreno en streaming,No aplica
4,La llama interior,Festival de Cine de Buenos Aires,Argentina


In [ ]:
!pip install geopy

import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep


df = pd.read_csv('/content/basededatos_visualizacion.csv', sep=';')

geolocator = Nominatim(user_agent="cine_chileno_mapper")


latitudes = []
longitudes = []


for lugar in df['País de estreno']: # Correct column name based on previous findings
    try:
        ubicacion = geolocator.geocode(lugar)
        if ubicacion:
            latitudes.append(ubicacion.latitude)
            longitudes.append(ubicacion.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
    except:
        latitudes.append(None)
        longitudes.append(None)
    sleep(2)  # Increased sleep time based on previous timeouts


df['lat'] = latitudes
df['lon'] = longitudes


df.to_csv('peliculas_festivales_coordenadas.csv', index=False)

print("✅ Coordenadas generadas y guardadas en 'peliculas_festivales_coordenadas.csv'")
df.head()

✅ Coordenadas generadas y guardadas en 'peliculas_festivales_coordenadas.csv'


,Película,Lugar de estreno,País de estreno,lat,lon
0,¿Quien tiene la culpa?,Estreno en salas,Chile,-31.761336,-71.318770
1,Patio de Chacales,Tallinn Black Nights Film Festival,Estonia,58.752378,25.331908
2,¿Cuándo te vas?,Estreno en salas,Chile,-31.761336,-71.318770
3,Desconectados 2: Reconectados,Estreno en streaming,No aplica,19.251484,-100.016774
4,La llama interior,Festival de Cine de Buenos Aires,Argentina,-34.996496,-64.967282


In [ ]:
!pip install altair_saver selenium pillow
!apt-get update
!apt install chromium-chromedriver

import os
os.environ["PATH"] += ":/usr/lib/chromium-browser/:/usr/lib/chromium-browser/chromedriver"

import pandas as pd
import altair as alt
from vega_datasets import data
import altair_saver


df = pd.read_csv('/content/peliculas_festivales_coordenadas.csv')

def asignar_continente(pais):
    america = ['Chile','Argentina','Brasil','México','Colombia','Perú','EE.UU.','Estados Unidos','Canadá']
    europa = ['Francia','España','Alemania','Italia','Reino Unido','Suiza','Suecia','Noruega','Portugal']
    asia = ['Japón','Corea del Sur','China','India']
    oceania = ['Australia','Nueva Zelanda']
    africa = ['Sudáfrica','Marruecos','Egipto']
    if pais in america: return 'América'
    elif pais in europa: return 'Europa'
    elif pais in asia: return 'Asia'
    elif pais in oceania: return 'Oceanía'
    elif pais in africa: return 'África'
    else: return 'Otro'

df['continente'] = df['País de estreno'].apply(asignar_continente)


df_agrupado = df.groupby(['País de estreno','lat','lon','continente']).agg(
    cantidad_peliculas=('Película ','count'),
    festivales=('Lugar de estreno', lambda x: ', '.join(sorted(set(x)))),
    peliculas=('Película ', lambda x: ', '.join(sorted(set(x))))
).reset_index()


sphere = alt.sphere()
graticule = alt.graticule()
source = alt.topo_feature(data.world_110m.url, 'countries')

base = alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='#e0f7fa'),
    alt.Chart(graticule).mark_geoshape(stroke='white', strokeWidth=0.5),
    alt.Chart(source).mark_geoshape(fill='#d8e3d8', stroke='white')
).project('naturalEarth1').properties(
    width=950, height=520, title='La huella internacional del cine chileno: países donde se estrena en festivales'
).configure_view(stroke=None)


puntos = alt.Chart(df_agrupado).mark_circle(opacity=0.85, stroke='black').encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=alt.Size('cantidad_peliculas:Q',
                  scale=alt.Scale(range=[100, 1200]),
                  legend=alt.Legend(title='Películas estrenadas')),
    color=alt.Color('continente:N',
                    scale=alt.Scale(domain=['América','Europa','Asia','Oceanía','África','Otro'],
                                    range=['#c62828','#283593','#00838f','#7b1fa2','#6d4c41','#616161']),
                    legend=alt.Legend(title='Región del festival')),
    tooltip=[
        alt.Tooltip('País de estreno:N', title='País del festival'),
        alt.Tooltip('cantidad_peliculas:Q', title='Películas chilenas estrenadas'),
        alt.Tooltip('festivales:N', title='Festivales'),
        alt.Tooltip('peliculas:N', title='Títulos')
    ]
).interactive()


mapa_final = base + puntos


mapa_final.save('mapa_festivales_interactivo.html')
altair_saver.save(mapa_final, 'mapa_festivales_interactivo.png', method='selenium', webdriver='chrome')

print("✅ Mapa narrativo guardado como HTML y PNG")
mapa_final

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

NoMatchingVersions: No matches for version='5.20.1' among ['4.0.2', '4.8.1', '4.17.0'].
Often this can be fixed by updating altair_viewer:
    pip install -U altair_viewer

In [ ]:
import altair_saver


mapa_final.save('/content/mapa_festivales_interactivo.html')


altair_saver.save(mapa_final, 'mapa_festivales_interactivo.png', method='selenium', webdriver='chrome')

print("✅ Visualización guardada como HTML y PNG")

NoMatchingVersions: No matches for version='5.20.1' among ['4.0.2', '4.8.1', '4.17.0'].
Often this can be fixed by updating altair_viewer:
    pip install -U altair_viewer

In [ ]:
!apt-get update
!apt install chromium-chromedriver

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os


chrome_options = Options()
chrome_options.add_argument("--headless") # Run in headless mode (without opening a browser window)
chrome_options.add_argument("--no-sandbox") # Required for running in Colab
chrome_options.add_argument("--disable-dev-shm-usage") # Required for running in Colab
chrome_options.add_argument("--disable-gpu") # Add this option
chrome_options.add_argument("--window-size=1920x1080") # Add this option

# Set the path to the chromedriver executable
# This should match the path where apt install chromium-chromedriver put it
webdriver_service = Service('/usr/lib/chromium-browser/chromedriver')

# Initialize the Chrome driver
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

# Path to the saved HTML file
html_file_path = '/content/mapa_festivales_interactivo.html'

# Open the HTML file in the browser
driver.get(f'file://{html_file_path}')

# Give the page a moment to render
time.sleep(2) # Adjust sleep time if needed for complex maps

# Save a screenshot of the page
driver.save_screenshot('mapa_festivales_html_screenshot.png')

# Close the browser
driver.quit()

print("✅ HTML map converted to mapa_festivales_html_screenshot.png")

WebDriverException: Message: Service /usr/lib/chromium-browser/chromedriver unexpectedly exited. Status code was: 1


In [ ]:

!pip install selenium pillow
!apt-get update
!apt install chromium-chromedriver

import os
from selenium import webdriver
from PIL import Image

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")


driver = webdriver.Chrome(options=chrome_options)


html_file = "/content/mapa_festivales_interactivo.html"
jpg_output = "mapa_festivales_interactivo.jpg"


driver.get("file://" + os.path.abspath(html_file))


driver.set_window_size(1200, 800)


driver.save_screenshot("temp_map.png")
driver.quit()


im = Image.open("temp_map.png").convert("RGB")
im.save(jpg_output, "JPEG", quality=95)

print("✅ Archivo HTML convertido y guardado como JPG:", jpg_output)


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
!pip install vl-convert-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 46.1 MB/s eta 0:00:00


In [ ]:
!pip install -U altair_saver altair altair_viewer

In [ ]:
!pip install -U altair_saver altair altair_viewer

In [ ]:
!pip install -U altair_viewer

In [ ]:
!pip install altair_saver selenium pillow
!apt-get update
!apt install chromium-chromedriver

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag